In [1]:
import pandas as pd
from quantrl_lab.data import (
    DataSourceRegistry,
    DataProcessor,
    IndicatorRegistry,
    AlpacaDataLoader, 
    YfinanceDataloader, 
    AlphaVantageDataLoader
)
from quantrl_lab.data.indicators.technical_indicators import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

#### What each data source supports (as of the latest implementation)

In [2]:
data_source_apc = AlpacaDataLoader()
supported_features = data_source_apc.supported_features
print(f"Supported features for AlpacaDataLoader: {supported_features}") 

data_source_yf = YfinanceDataloader()
supported_features = data_source_yf.supported_features
print(f"Supported features for YfinanceDataLoader: {supported_features}")

data_source_av = AlphaVantageDataLoader()
supported_features = data_source_av.supported_features
print(f"Supported features for AlphaVantageDataLoader: {supported_features}")

Supported features for AlpacaDataLoader: ['historical_bars', 'news', 'live_data', 'streaming']
Supported features for YfinanceDataLoader: ['historical_bars']
Supported features for AlphaVantageDataLoader: ['historical_bars', 'news']


#### The technical indicators that are supported as of now (more can be added later on)

In [3]:
print(f"Available indicators:  {IndicatorRegistry.list_all()}")

Available indicators:  ['SMA', 'EMA', 'RSI', 'MACD', 'ATR', 'BB', 'STOCH', 'OBV']


In [4]:
data_loader = DataSourceRegistry()

In [5]:
olhcv_df = data_loader.get_historical_ohlcv_data(
    symbols="MU",
    start="2023-01-01",
    end="2025-06-30",
    timeframe="1d",
)

Fetching historical data for MU from 2023-01-01 to 2025-06-30 with timeframe 1d

In [6]:
olhcv_df.head()

,Symbol,Timestamp,Open,High,Low,Close,Volume,Trade_count,VWAP,Date
0,MU,2023-01-03 05:00:00+00:00,50.56,50.960,49.55,50.37,14357184.0,126221.0,50.323799,2023-01-03
1,MU,2023-01-04 05:00:00+00:00,53.05,54.520,52.73,54.20,27565757.0,208706.0,53.718035,2023-01-04
2,MU,2023-01-05 05:00:00+00:00,53.89,55.390,53.89,54.71,21227741.0,153295.0,54.692799,2023-01-05
3,MU,2023-01-06 05:00:00+00:00,55.43,57.290,54.03,56.77,23910501.0,173995.0,56.297094,2023-01-06
4,MU,2023-01-09 05:00:00+00:00,56.82,57.515,55.65,56.36,17380586.0,166496.0,56.625176,2023-01-09


In [7]:
news_df = data_loader.get_news_data(
    "MU",
    start="2023-01-01",
    end="2025-06-30",
)

Output()

✓ Total news items fetched: 1648

In [8]:
data_processor = DataProcessor(olhcv_data=olhcv_df, news_data=news_df)

# configuring the technical indicators with specific parameters
processed_data = data_processor.data_processing_pipeline(
    indicators=["SMA", "EMA", "RSI", "MACD", "ATR", "BB", "STOCH", "OBV"],
    # The following are optional, you could do
    # data_processor.append_technical_indicators(olhcv_df, indicators=["SMA", "EMA"])
    # and the default parameters will be used.
    
    # fillna_strategy="neutral",
    SMA_params={
        "window": 20,
    },
    EMA_params={"window": 9},
    RSI_params={"window": 7},
    MACD_params={"fast": 12, "slow": 26, "signal": 9},
    ATR_params={"window": 14},
    BB_params={"window": 20, "num_std": 2},
    STOCH_params={"k_window": 14, "d_window": 3, "smooth_k": 1},
    OBV_params={},
)

Applying SMA with params: {'window': 20}

Applying EMA with params: {'window': 9}

Applying RSI with params: {'window': 7}

Applying MACD with params: {'fast': 12, 'slow': 26, 'signal': 9}

Applying ATR with params: {'window': 14}

Applying BB with params: {'window': 20, 'num_std': 2}

Applying STOCH with params: {'k_window': 14, 'd_window': 3, 'smooth_k': 1}

Applying OBV with params: {}

Device set to use cpu


✓ Sentiment analysis pipeline initialized with model: ProsusAI/finbert

In [9]:
processed_data.head()

,Open,High,Low,Close,Volume,Trade_count,VWAP,SMA_20,EMA_9,RSI_7,MACD_line_12_26,MACD_signal_9,MACD_histogram,ATR_14,BB_middle_20,BB_upper_20_2,BB_lower_20_2,BB_bandwidth_20,STOCH_%K_14,STOCH_%D_3,OBV,sentiment_score
0,59.10,60.34,58.6500,60.30,27000145.0,199847.0,59.869803,58.1680,60.708337,56.319683,2.619337,2.408994,0.210343,2.099526,58.1680,64.819805,51.516195,0.228710,55.408389,73.730684,53231005.0,0.966646
1,60.62,63.23,60.3850,62.60,24275883.0,186598.0,61.915838,58.7795,61.086669,66.379157,2.628630,2.452922,0.175708,2.158846,58.7795,64.610923,52.948077,0.198417,80.794702,69.058131,77506888.0,0.914797
2,63.56,63.92,61.8600,63.17,16087925.0,160979.0,62.950759,59.2280,61.503335,68.478078,2.651424,2.492622,0.158802,2.151785,59.2280,64.955270,53.500730,0.193397,87.086093,74.429728,93594813.0,0.000000
3,62.00,63.79,61.9000,62.41,14142431.0,110665.0,62.757956,59.6130,61.684668,62.416669,2.578441,2.509786,0.068655,2.133086,59.6130,65.091302,54.134698,0.183796,78.697572,82.192789,79452382.0,0.000000
4,60.94,61.18,59.7401,59.98,13316275.0,111271.0,60.216354,59.7735,61.343735,46.923190,2.298030,2.467435,-0.169404,2.171430,59.7735,65.086698,54.460302,0.177778,51.876380,72.553348,66136107.0,0.000000
